# Find Days To Use for Lineup Optimization
This notebook has code that will list statistics for games days for a season of a sport. Use this to figure out which days to use as test cases for optimizing lineup generation settings.

In [ ]:
import os
from sqlalchemy import func
import qgrid
from IPython import display
from pandas import DataFrame

from fantasy_py import db

db_file = "/home/delano/working/fantasy/mlb_hist_2008-2018.db"
seasons = (2016, 2017, 2018)

os.chdir(os.environ['FANTASY_HOME'])
db_obj = db.get_db_obj(db_file)

data = {'date': [],
        'games': []}

with db_obj.session_scoped() as session:
    for game_date, count in session.query(db.Game.date, func.count()) \
                                   .filter(db.Game.season.in_(seasons)) \
                                   .group_by(db.Game.date) \
                                   .order_by(db.Game.date) \
                                   .all():
        data['date'].append(game_date)
        data['games'].append(count)

    
df = DataFrame.from_dict(data)
# note that the number of players per game is uniquely defined by sport
if db_obj.db_manager.ABBR == 'mlb':
    df['players'] = df['games'] * 9
else:
    raise NotImplemented("do something for this sport's player count")

display.display(qgrid.QgridWidget(df=df))